In [3]:
import pandas as pd
import numpy as np
import altair as alt

alt.themes.enable('vox')

ThemeRegistry.enable('vox')

In [4]:
apr18650_df = pd.read_csv('plots_data/capacity_fading_apr18650m1.csv')
anr26650_df = pd.read_csv('plots_data/capacity_fading_anr26650m1a.csv')
anr26650_df.rename({'25_deg_x': 'x', '25_deg_y': 'y'}, axis=1, inplace=True)
anr26560_df = anr26650_df.iloc[:, 0:2].dropna()
apr18650_df['battery'] = 'A123 APR18650M1'
anr26650_df['battery'] = 'A123 ANR26650M1A'

In [5]:
alt.Chart(pd.concat([apr18650_df, anr26650_df])).encode(
    x=alt.X('x:Q', title='Cycles (n)'),
    y=alt.Y('y:Q', scale=alt.Scale(
        domain=[80, 100], zero=False), title='Discharge capacity (%)'),
    color=alt.Color('battery:N',
                    scale=alt.Scale(
                        domain=['A123 ANR26650M1A', 'A123 APR18650M1'], scheme='set1'),
                    legend=alt.Legend(title='Battery type'),
                    )

).mark_line().transform_filter(alt.datum.x < 1000).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_view(
    width=600
)

alt.Chart(...)

In [152]:
import os
import shutil
from OMPython import OMCSessionZMQ, ModelicaSystem

shutil.rmtree('/home/midren/bachelor/tmp')
os.mkdir('/home/midren/bachelor/tmp')
os.chdir('/home/midren/bachelor/tmp')
model = ModelicaSystem("/home/midren/bachelor/ModelicaSrc/TheveninBasedBattery.mo","TheveninBasedBattery",["Modelica"])
os.chdir('/home/midren/bachelor')

2021-03-19 10:01:20,425 - OMPython - INFO - OMC Server is up and running at file:////tmp/openmodelica.midren.port.2926750cfce74fe29658e575b0ab8562 pid=607501


Notification: Automatically loaded package Modelica 3.2.3 due to uses annotation.
Notification: Automatically loaded package Complex 3.2.3 due to uses annotation.
Notification: Automatically loaded package ModelicaServices 3.2.3 due to uses annotation.

Expected end of text, found '-'  (at char 192), (line:1, col:193)


In [140]:
from pathlib import Path

charts = dict()

def get_soc_dependence(model, variable, is_charging=True):
    os.chdir('/home/midren/bachelor/tmp')
    model.setSimulationOptions(["startTime=0", "stopTime=7200", "stepSize=100"])
    model.simulate()
#     "voltageSource.U_ocv.v"
    var, soc, t = model.getSolutions([variable, "capacityFadingCalc.SoC", "time"])
    
    st = np.argmin(np.abs(t - 3600)) if is_charging else 0
    en = np.argmin(np.abs(t - 7200)) if is_charging else np.argmin(np.abs(t - 3600))
    simulation_res = pd.DataFrame({"SoC": soc[st:en], variable: var[st:en]})
    simulation_res['type'] = 'simulation'
    os.chdir('/home/midren/bachelor')
    return simulation_res

def get_param_charts(is_charging):
    modifier = '_charging' if is_charging else '_discharging'
    for filename in sorted(Path(".").glob('plots_data/thevenin_params/*' + modifier + '.csv')):
        df = pd.read_csv(filename)
        def remove_modifier(x): return "_".join(x.split('_')[:-1])
        component = remove_modifier(filename.stem)
        component_type = remove_modifier(component)
        variable = ".".join([component, component_type])
        sim_df = get_soc_dependence(model, variable, is_charging).rename({variable: component}, axis=1)
        df.rename(columns={'x': 'SoC', 'y': component}, inplace=True)
        df.sort_values('SoC', inplace=True)
        df['type'] = 'paper'
        unit = 'Ohm' if component_type == 'R' else 'Farad'
        chrt = alt.Chart(df.append(sim_df)).encode(
            x=alt.X('SoC:Q', title='SoC'),
            y=alt.Y(field=component, type='quantitative', title=component_type + '(' + unit +')', scale=alt.Scale(zero=True)),
            color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
        ).mark_line(
        ).transform_filter(
            alt.datum.SoC > 0.05
        ).properties(
            title=component
#             title=modifier[1:]
        )


        charts[component] = chrt
    return charts
        
charts = get_param_charts(not True)

alt.vconcat(
    charts['R_s'],
    (charts['R_ts'] | charts['C_ts']),
    (charts['R_tl'] | charts['C_tl']),
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_title(
    font='CMU Serif',
    fontSize=17
)


alt.VConcatChart(...)

In [158]:
simulation_u_ocv = get_soc_dependence(model, "voltageSource.U_ocv.v").rename({'voltageSource.U_ocv.v': 'U_ocv'}, axis=1)
paper_u_ocv = pd.read_csv('plots_data/u_ocv.csv').rename({"x": 'SoC', 'y': 'U_ocv'}, axis=1)
paper_u_ocv['type'] = 'paper'

alt.Chart(simulation_u_ocv.append(paper_u_ocv)).mark_line().encode(
    x = alt.X('SoC:Q'),
    y = alt.Y('U_ocv:Q', scale=alt.Scale(zero=False), title='U_ocv(V)'),
    color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
).configure_view(
    height=300, 
    width=600
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
)


alt.Chart(...)

In [157]:
test_df = pd.read_csv('plots_data/test_cycle.csv')
test_df['x'] = test_df['x'].apply(lambda x: max(x, 0))
# test_df['y'] = test_df['y'].apply(lambda x: round(x, 0))
# test_df = test_df.loc[test_df['y'].shift() != test_df['y']]

alt.Chart(test_df).mark_line(interpolate='step').encode(
    x = alt.X('x:Q', title='time(min)'),
    y = alt.Y('y:Q', scale=alt.Scale(zero=False), title='I_req(A)'),
    color=alt.value('red')
#     color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
).configure_view(
    height=300, 
    width=800
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
)

alt.Chart(...)

In [146]:
current_table = "[" + "; ".join([ str(round(row['x']*60, 2)) +', ' + str(round(row['y']*1.1/3600,6)) for i, row in test_df.iterrows()]) + "]"

In [147]:
alt.Chart(simulation_res).mark_line().encode(
    x = alt.X('time:Q'),
    y = alt.Y('voltage:Q', scale=alt.Scale(zero=False)),
    color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
).configure_view(
    height=300, 
    width=600
).configure_legend(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
).configure_axis(
    labelFont='CMU Serif',
    titleFont='CMU Serif',
    labelFontSize=15,
    titleFontSize=15,
)

alt.Chart(...)

In [161]:
shutil.rmtree('/home/midren/bachelor/tmp2')
os.mkdir('/home/midren/bachelor/tmp2')
os.chdir('/home/midren/bachelor/tmp2')
model = ModelicaSystem("/home/midren/bachelor/ModelicaSrc/TestCycleTheveninBasedBattery.mo","TestCycleTheveninBasedBattery",["Modelica"])
os.chdir('/home/midren/bachelor')

2021-03-19 10:03:37,083 - OMPython - INFO - OMC Server is up and running at file:////tmp/openmodelica.midren.port.1fc3b70130024f8d970d894657c0fc9d pid=616649


Notification: Automatically loaded package Modelica 3.2.3 due to uses annotation.
Notification: Automatically loaded package Complex 3.2.3 due to uses annotation.
Notification: Automatically loaded package ModelicaServices 3.2.3 due to uses annotation.

Expected end of text, found '-'  (at char 192), (line:1, col:193)


In [162]:
def get_test_voltage_response(is_charging):
    os.chdir('/home/midren/bachelor/tmp2')
    model.setSimulationOptions(["startTime=0", "stopTime=10000", "stepSize=100"])
    model.simulate()
    #     "voltageSource.U_ocv.v"
    var, soc, t = model.getSolutions(["voltageSensor.v", "coulombSocCounter.SoC", "time"])
    os.chdir('/home/midren/bachelor')

    st = np.argmin(np.abs(t - 6500)) if is_charging else 0
    en = np.argmin(np.abs(t - 10000)) if is_charging else np.argmin(np.abs(t - 3800))
    time = t[st:en]/60-109 if is_charging else t[st:en]/60
    simulation_res = pd.DataFrame({"time": time, "voltage": var[st:en]})
    simulation_res['type'] = 'simulation'
    modifier = ("" if is_charging else "dis") + "charging"
    measurement_output_df = pd.read_csv('plots_data/measurement_' + modifier + '.csv').rename(columns={'x': 'time', 'y': 'voltage'})
    measurement_output_df['type'] = 'measurement'

    return alt.Chart(simulation_res.append(measurement_output_df)).mark_line().encode(
        x = alt.X('time:Q', title='time(min)'),
        y = alt.Y('voltage:Q', scale=alt.Scale(zero=False), title='U_bat(V)'),
        color=alt.Color('type:N', scale=alt.Scale(scheme='set1'))
    )

In [165]:
alt.vconcat(
    get_test_voltage_response(False).properties(
        title='charging'
    ),
    get_test_voltage_response(True).properties(
    title='discharging'
)).configure_view(
        height=300, 
        width=600
    ).configure_legend(
        labelFont='CMU Serif',
        titleFont='CMU Serif',
        labelFontSize=15,
        titleFontSize=15,
    ).configure_axis(
        labelFont='CMU Serif',
        titleFont='CMU Serif',
        labelFontSize=15,
        titleFontSize=15,
) 

alt.VConcatChart(...)